## Understanding __delattr__

__delattr__(self,name) method is always called when there is an attempt to delete an attribute through the del statement. Python implicitly passes the name of the attribute need to be delted in string format to __delattr__ method. 

In below example when del is applied on x attribute of object 

In [1]:
class Test:
    # called when del applied an attribute 
    def __delattr__(self,name):
        print('inside __delattr__')
        super().__delattr__(name)
        print (f'{name} deleted.')
        

T1 = Test()
T1.x = 10
del T1.x


inside __delattr__
x deleted.


In [2]:
"""
In below example __delattar__ method is used to make x and y attribute of class object
Point read only. An attemt to delete attributes will invoke __delattar__ method which 
raises AttributeError exception.
"""
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y

#delattr called whenever attribute of point is deleted 
    def __delattr__(self,name):
        #raises attribute error if x or y attribute is deleted.
        if name in('x','y'):
            raise AttributeError(f'Point attribute {name} is read only')

P = Point(1,2)
del P.x

AttributeError: Point attribute x is read only

### Understanding setattr:

In [ ]:

"""
Understanding __setattr__:

__setattr__(name,value) method is always called when there is an attempt to set the 
attribute value through dot notation obj.name = value or through setattr(obj,'name',value)  __setattr__ method invoked. 

For the below example   we can see whenever an attribute value changed through 
dot notation setattr or __setattr__ is called with the attribute name and corrosponding value.inside __setattr__ the value of an attribute is changed by 
passing received parameters to the object class  __setattr__ method. Value can also
be set  by assigning value to object dictionary self.__dict__[name] = value.  
"""

In [2]:
"""
Note: to set value with __setattr__ method you need to invoke super method. else
calling like below will invoke infinite recursion. 
 class Test:
    #__setattr__ infinite recursion
    def __setattr__(self,name,value):
        self.name=value


"""

class Test:
    # ___setattr__ invoked everytime value is set.class
    def __setattr__(self,name,value):
        print(f'inside __setattr__ name: {name}; value:{value}')
        #set the value through super()
        super().__setattr__(name,value)

T1 = Test()

T1.x = 10                      
setattr(T1,'y',20)

T1.__dict__['x'] = 100   # Note: here we directly set the value in instance __dict__. So, setattr will not run

print(f'T1.x = {T1.x}; T1.y={T1.y}')


inside __setattr__ name: x; value:10
inside __setattr__ name: y; value:20
T1.x = 100; T1.y=20


In [21]:
"""
__setattr__ method used to implement the Constant class which does not allow to 
attribute value to change once created.
"""
class Constant:
    def __init__(self,value):
        super().__setattr__('value',value)
    
    def __setattr__(self,name,value):
        raise AttributeError("Constant value can't be changed")

pi = Constant(3.14)
pi.value = 1.5


AttributeError: Constant value can't be changed

In [35]:
import logging,datetime

logging.basicConfig(level=logging.DEBUG)

class Logchange:
    def __init__(self,c,nc):
        self.critical = c
        self.noncritical = nc
    
        def __setattr__(self,name,value):
            t = datetime.datetime.now().time()
            logging.debug(f'updated {name} with value {value} @ {t}')
            super().__setattr__(name,value)

T1 = Logchange(111,1)

T1.critical = 777
T1.noncritical = 100
T1.critical  = 999

## Understanding __getattr__
____getattr____(self,name) method of a class is called if an attribute by name cannot 
found in an object or in class and  its superclass. ____getattr____ method is invoked 
whenever the nonexisting attribute is accessed through dot notation(obj.name) or
getattr(obj,'name') or hasattr(obj,'name')

In below example ____getattr____ method is implemented inside Test class.
- When attribute x is accessed class attribute x is returned.
- When attribute y is accessed class attribute y is returned.
- When attributes z,a,b which is not present in object or class are accessed 
  through dot notation, getattr(),hasattr() respectively ____getattr____() 
  method is invoked with the attribute name. 

  

In [ ]:
class Person:
    age = 23
    name = "Adam"

person = Person()

# when default value is provided
print('The sex is:', getattr(person, 'sex', 'Male'))

# when no default value is provided
print('The sex is:', getattr(person, 'sex'))

In [5]:
class Test:
    x = 100
    def __init__(self,y):
        self.y = y
    def __getattr__(self,name):
        print('inside getattr',end=':') 
        print('attribute {!r} not found'.format(name))

T1 = Test(200)
print(f'T1.x = {T1.x}')
print(f'T1.y = {T1.y}')
T1.z
getattr(T1,'a')
hasattr(T1,'b')

T1.x = 100
T1.y = 200
inside getattr:attribute 'z' not found
inside getattr:attribute 'a' not found
inside getattr:attribute 'b' not found


True

In [23]:
class DefaultNone:
    def __getattr__(Self,name):
        if name.isalpha():
            super().__setattr__(name,None)
        else:
            raise AttributeError('{name} should be in alpha only')

D = DefaultNone()
D.x = 'abc'
D.y

### Understanding ____getattribute____

In [ ]:
"""
__getattribute__(self,name) is always called regardless attribute exists or not when there is an attempt to retrive the named attribute, except when the attribute is 
special attribute or method (special method lookup). Dot notation (obj.name) or the
getattr() or hasattr() triggers __getattribute__ method.To retrive attributes of the
isinstance obj without triggering an infinite recursion, 
__getattribute__ should use super().__getattribute__(obj,name)
"""

In [29]:
class Test:
    def __getattribute__(self,name):
        print(f'Getting attribute {name}.',end=':')
        return super().__getattribute__(name)

T1 = Test()
T1.x = 100
T1.y = 200
T1.z = 300
 
print(T1.x)
print(getattr(T1,'y'))
print(hasattr(T1,'z'))



Getting attribute x.:100
Getting attribute y.:200
Getting attribute z.:True


In [32]:
"""
__getattr__ is invoked only after __getattribute__, and only when __getattribute__ raises Attribute error.
.In below example since __getattribute__ do not raise any AttributeError even though Nonexisting attribute 
x and y are accessed __getattr__ method is not called.
"""


class Test1:
    def __getattribute__(self,name):
        print(f'inside __getattribute__ {name}')
    
    def __getattr__(self,name):
        print(f'inside __getattr__ {name}')

T1 = Test1()
T1.x
T1.y
T1.z = 12

inside __getattribute__ x
inside __getattribute__ y
inside __getattribute__ z


In [7]:
"""
In below example __getattribute__ method is implemented to hide the credit card number when accessed directly. Whenever a credit card number is accessed __getattribute__ replace the first 12 digits with X.
"""

class Creditcard:
    def __init__(self,cardno,amount):
        self.number =  cardno
        self.amount =  amount

    def __getattribute__(self,name):
        if name == 'number':
            return 'XXX-'*3+super().__getattribute__(name)[-4:]
        else:
            return super().__getattribute__(name)


C1 = Creditcard('474837483993',1000)
print(C1.number)
print(C1.amount)

XXX-XXX-XXX-3993
1000
